# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, cross_val_score

## Откройте и изучите файл

In [2]:
df=pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


**на входе имеем таблицу 3214х5, данные в столбцах полные, типы данных уже приведены к необходимым.**

сalls — количество звонков,

minutes — суммарная длительность звонков в минутах,

messages — количество sms-сообщений,

mb_used — израсходованный интернет-трафик в Мб,

is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Разбейте данные на выборки

In [5]:
#Разобьем исходный датафрейм на 2 части, features и target,
#где features это признаки тарифа(звонки смс трафик), а target значение is_ultra
#каждую разобьем на обучающую и валидационную, размер валидационной выборки составит 25%
features = df.drop('is_ultra', axis=1)
target = df.is_ultra
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345
)

**Думал сделать стратифицированную выборку, но решил что на этом проекте это не так категорично**

## Исследуйте модели

In [6]:

tree = DecisionTreeClassifier(max_depth=9, random_state=12345)
tree_params = {'max_depth': range(1,11),
'max_features': range(1,4)}
tree_grid = GridSearchCV(tree, tree_params,
cv=5)

In [7]:
tree_grid.fit(features_train, target_train)

GridSearchCV(cv=5,
             estimator=DecisionTreeClassifier(max_depth=9, random_state=12345),
             param_grid={'max_depth': range(1, 11),
                         'max_features': range(1, 4)})

In [8]:
tree_grid.best_params_, tree_grid.best_score_

({'max_depth': 3, 'max_features': 3}, 0.7970954356846474)

In [9]:
#используем кросс_валидацию для выбора наилучших параметров модели случайныый лес
forest = RandomForestClassifier(n_estimators=100, random_state=17)
forest_params = {'max_depth': range(1,11),
'max_features': range(1,4)}
forest_grid = GridSearchCV(forest, forest_params,
cv=5)
forest_grid.fit(features_train, target_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=17),
             param_grid={'max_depth': range(1, 11),
                         'max_features': range(1, 4)})

In [10]:
forest_grid.best_params_, forest_grid.best_score_

({'max_depth': 10, 'max_features': 3}, 0.8153526970954357)

## Проверьте модель на тестовой выборке

In [11]:
forest_grid.score(features_valid, target_valid)

0.7985074626865671

In [12]:
#попали в целевое значение более 75%

## (бонус) Проверьте модели на адекватность

In [13]:
accuracy_score(target_valid, tree_grid.predict(features_valid))

0.7885572139303483

In [ ]:
##добавил roc auc

In [18]:
roc_auc_score(target_valid, tree_grid.predict(features_valid))

0.6864603524391413

In [16]:
knn = KNeighborsClassifier(algorithm='auto', weights='uniform')
knn_params = {'n_neighbors': range(1, 10)}
knn_grid = GridSearchCV(knn, knn_params,
cv=5, n_jobs=-1,
verbose=True)
knn_grid.fit(features_train, target_train)
knn_grid.best_params_, knn_grid.best_score_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


({'n_neighbors': 8}, 0.7589211618257261)

**Как то так получается :)**

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
